In [1]:
from llm_manager import (
    SequenceClassifierOutputOverRanges,
    EmbeddingBasedClassifier,
    ClassifierBase,
    ID2LABEL,
    LABEL2ID,
    MODEL_NAME,
)
from dataset_manager import ROOT, MovieLensManager, INPUT_EMBEDS_REPLACE_KGE_DIMENSION
from graph_representation_generator import GraphRepresentationGenerator
from typing import Optional, Union, Tuple
import os

import torch
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
from transformers import BertForSequenceClassification

# Customize Input Embeds Replace Classifier
In our first experiments we noticed that replacing the placeholders of the input embeds with the KGEs breakes up the gradient propagation all the way back. To make it even, we will freeze this entire step so that only the attention headers are trained. 

In [2]:
class GraphPrompterHFFrozenBertForSequenceClassification(
    BertForSequenceClassification
):
    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        graph_embeddings: Optional[torch.Tensor] = None,
        token_type_ranges: Optional[torch.Tensor] = None,
    ) -> Union[Tuple[torch.Tensor], SequenceClassifierOutputOverRanges]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
            Labels for computing the sequence classification/regression loss. Indices should be in `[0, ...,
            config.num_labels - 1]`. If `config.num_labels == 1` a regression loss is computed (Mean-Square loss), If
            `config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
        return_dict = (
            return_dict if return_dict is not None else self.config.use_return_dict
        )
        if inputs_embeds is None:
            # CHANGES: we freeze the gradients when producing input embeddings and detach them before passing them on just to be save
            with torch.no_grad():
                inputs_embeds = self.bert.embeddings(input_ids).detach()
            assert isinstance(inputs_embeds, torch.Tensor)
        if graph_embeddings is not None and len(graph_embeddings) > 0:
            if attention_mask is not None:
                mask = (
                    (
                        (attention_mask.to(self.device).sum(dim=1) - 1)
                        .unsqueeze(1)
                        .repeat((1, 2))
                        - torch.tensor([3, 1], device=self.device)
                    )
                    .unsqueeze(2)
                    .repeat((1, 1, self.config.hidden_size))
                )  # basically a mask finding the last positions between the sep tokens (reshaped so they can be used in scatter)
                inputs_embeds = inputs_embeds.to(
                    self.device
                ).scatter(
                    1, mask.to(self.device), graph_embeddings.to(self.device)
                )  # replace the input embeds at the place holder positions with the KGEs.
        outputs = self.bert(
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )  # feed forward the input embeds to the attention model

        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (
                    labels.dtype == torch.long or labels.dtype == torch.int
                ):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                if self.num_labels == 1:
                    loss = loss_fct(logits.squeeze(), labels.squeeze())
                else:
                    loss = loss_fct(logits, labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)
        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutputOverRanges(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
            token_type_ranges=token_type_ranges,
        )

Next we define the new Classifier the same way the original was defined, with the exception of the model we are loading, which is the new frozen model.

In [3]:
class GraphPrompterHFFrozenClassifier(EmbeddingBasedClassifier):
    def __init__(
        self,
        kge_manager,
        get_embedding_cb,
        root_path,
        model_name=MODEL_NAME,
        model_max_length=256,
        false_ratio=1.0,
        force_recompute=False,
    ) -> None:
        training_path = f"{root_path}/training"
        model_path = f"{training_path}/best"

        if os.path.exists(model_path) and not force_recompute:
            model = (
                GraphPrompterHFFrozenBertForSequenceClassification.from_pretrained(
                    model_path,
                    num_labels=2,
                    id2label=ID2LABEL,
                    label2id=LABEL2ID,
                )
            )
        else:
            model = (
                GraphPrompterHFFrozenBertForSequenceClassification.from_pretrained(
                    model_name, num_labels=2, id2label=ID2LABEL, label2id=LABEL2ID
                )
            )
        assert isinstance(model, BertForSequenceClassification)
        super().__init__(
            kge_manager,
            get_embedding_cb,
            root_path,
            model,
            model_name,
            model_max_length,
            false_ratio,
            force_recompute,
        )

    def plot_training_loss_and_accuracy(self):
        model_type = "Input Embeds Replace"
        self._plot_training_loss_and_accuracy(model_type)

Now comes the whole training procedure (see training_models.ipynb)

In [4]:
kg_manager = MovieLensManager()
EPOCHS = 20
BATCH_SIZE = 256
graph_representation_generator_graph_prompter_hf = GraphRepresentationGenerator(
    kg_manager.data,
    kg_manager.gnn_train_data,
    kg_manager.gnn_val_data,
    kg_manager.gnn_test_data,
    hidden_channels=INPUT_EMBEDS_REPLACE_KGE_DIMENSION,
    kge_dimension=INPUT_EMBEDS_REPLACE_KGE_DIMENSION,
)
graph_prompter_hf_embeddings = (
    graph_representation_generator_graph_prompter_hf.get_saved_embeddings(
        "graph_prompter_hf"
    )
)
save = False
if graph_prompter_hf_embeddings is None:
    graph_prompter_hf_embeddings = (
        graph_representation_generator_graph_prompter_hf.generate_embeddings(
            kg_manager.llm_df
        )
    )
    save = True
kg_manager.append_graph_prompter_hf_graph_embeddings(
    graph_prompter_hf_embeddings, save=save
)

loading pretrained model
Device: 'cuda'


In [5]:
INPUT_EMBEDS_REPLACE_FROZEN_ROOT = (
    f"{ROOT}/llm/graph_prompter_hf_frozen"  # make sure the dir exists
)
if not os.path.exists(INPUT_EMBEDS_REPLACE_FROZEN_ROOT):
    os.makedirs(INPUT_EMBEDS_REPLACE_FROZEN_ROOT)
graph_prompter_hf_frozen_bert_classifier = GraphPrompterHFFrozenClassifier(
    kg_manager,
    graph_representation_generator_graph_prompter_hf.get_embedding,
    root_path=INPUT_EMBEDS_REPLACE_FROZEN_ROOT,
)
dataset_graph_prompter_hf = (
    kg_manager.generate_graph_prompter_hf_embedding_dataset(
        graph_prompter_hf_frozen_bert_classifier.tokenizer.sep_token,
        graph_prompter_hf_frozen_bert_classifier.tokenizer.pad_token,
        graph_prompter_hf_frozen_bert_classifier.tokenize_function,
        force_recompute=False,
    )
)
dataset_graph_prompter_hf

Some weights of GraphPrompterHFFrozenBertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DatasetDict({
    train: Dataset({
        features: ['source_id', 'target_id', 'id_x', 'id_y', 'prompt_feature_title', 'prompt_feature_genres', 'labels', 'split', 'prompt', 'gnn_feature_(no genres listed)', 'gnn_feature_Action', 'gnn_feature_Adventure', 'gnn_feature_Animation', 'gnn_feature_Children', 'gnn_feature_Comedy', 'gnn_feature_Crime', 'gnn_feature_Documentary', 'gnn_feature_Drama', 'gnn_feature_Fantasy', 'gnn_feature_Film-Noir', 'gnn_feature_Horror', 'gnn_feature_IMAX', 'gnn_feature_Musical', 'gnn_feature_Mystery', 'gnn_feature_Romance', 'gnn_feature_Sci-Fi', 'gnn_feature_Thriller', 'gnn_feature_War', 'gnn_feature_Western', 'graph_prompter_hf_source_embedding', 'graph_prompter_hf_target_embedding', 'graph_embeddings', 'input_ids', 'attention_mask', 'token_type_ranges'],
        num_rows: 56469
    })
    val: Dataset({
        features: ['source_id', 'target_id', 'id_x', 'id_y', 'prompt_feature_title', 'prompt_feature_genres', 'labels', 'split', 'prompt', 'gnn_feature_(no gen

In [6]:
graph_prompter_hf_frozen_bert_classifier.train_model_on_data(
    dataset_graph_prompter_hf, epochs=EPOCHS, batch_size=BATCH_SIZE
)
# now we don't have to generate new non-existing edges, because they were generated in the beginning anyways.
graph_prompter_hf_frozen_bert_classifier = GraphPrompterHFFrozenClassifier(
    kg_manager,
    graph_representation_generator_graph_prompter_hf.get_embedding,
    root_path=INPUT_EMBEDS_REPLACE_FROZEN_ROOT,
    false_ratio=-1.0,  # init with false_ratio of -1 so no new false edges are produced on the fly
)
graph_prompter_hf_frozen_df = (
    graph_prompter_hf_frozen_bert_classifier.forward_dataset_and_save_outputs(
        dataset_graph_prompter_hf,
        kg_manager.get_vanilla_tokens_as_df,
        epochs=1,
        batch_size=BATCH_SIZE,
        force_recompute=True,
    )
)

  0%|          | 0/4420 [00:00<?, ?it/s]

c:\Users\MARS\.conda\envs\hauptprojekt\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{'loss': 0.7055, 'grad_norm': 0.6920956969261169, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.05}
{'loss': 0.7042, 'grad_norm': 1.1834245920181274, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.09}
{'loss': 0.7011, 'grad_norm': 0.6951578855514526, 'learning_rate': 3e-06, 'epoch': 0.14}
{'loss': 0.7024, 'grad_norm': 0.6460822820663452, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.18}
{'loss': 0.7003, 'grad_norm': 0.737121045589447, 'learning_rate': 5e-06, 'epoch': 0.23}
{'loss': 0.6973, 'grad_norm': 1.1496777534484863, 'learning_rate': 6e-06, 'epoch': 0.27}
{'loss': 0.6954, 'grad_norm': 0.5198498368263245, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.32}
{'loss': 0.695, 'grad_norm': 0.4533461630344391, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.36}
{'loss': 0.6926, 'grad_norm': 0.4349064230918884, 'learning_rate': 9e-06, 'epoch': 0.41}
{'loss': 0.6885, 'grad_norm': 0.5290090441703796, 'learning_rate': 1e-05, 'epoch': 0.45}
{'loss': 0.6837, 'grad_norm': 

  0%|          | 0/134 [00:00<?, ?it/s]

{'eval_loss': 0.5785201787948608, 'eval_accuracy': 0.7287947730719869, 'eval_runtime': 31.0727, 'eval_samples_per_second': 1103.347, 'eval_steps_per_second': 4.312, 'epoch': 1.0}
{'loss': 0.6016, 'grad_norm': 0.6119468212127686, 'learning_rate': 2.3000000000000003e-05, 'epoch': 1.04}
{'loss': 0.6047, 'grad_norm': 0.5500653386116028, 'learning_rate': 2.4e-05, 'epoch': 1.09}
{'loss': 0.5903, 'grad_norm': 0.7359963059425354, 'learning_rate': 2.5e-05, 'epoch': 1.13}
{'loss': 0.5866, 'grad_norm': 0.9616073369979858, 'learning_rate': 2.6000000000000002e-05, 'epoch': 1.18}
{'loss': 0.5692, 'grad_norm': 0.8594094514846802, 'learning_rate': 2.7000000000000002e-05, 'epoch': 1.22}
{'loss': 0.5854, 'grad_norm': 0.8118565082550049, 'learning_rate': 2.8000000000000003e-05, 'epoch': 1.27}
{'loss': 0.5623, 'grad_norm': 0.7552378177642822, 'learning_rate': 2.9e-05, 'epoch': 1.31}
{'loss': 0.567, 'grad_norm': 0.5573002696037292, 'learning_rate': 3e-05, 'epoch': 1.36}
{'loss': 0.5685, 'grad_norm': 0.5983

...

{'eval_loss': 0.24998049437999725, 'eval_accuracy': 0.8953447672383619, 'eval_runtime': 30.5257, 'eval_samples_per_second': 1123.121, 'eval_steps_per_second': 4.39, 'epoch': 19.0}
{'loss': 0.2799, 'grad_norm': 0.9792028069496155, 'learning_rate': 2.806122448979592e-06, 'epoch': 19.0}
{'loss': 0.2888, 'grad_norm': 0.9682706594467163, 'learning_rate': 2.6785714285714285e-06, 'epoch': 19.05}
{'loss': 0.2852, 'grad_norm': 0.8230661749839783, 'learning_rate': 2.5510204081632653e-06, 'epoch': 19.1}
{'loss': 0.2655, 'grad_norm': 0.9372957348823547, 'learning_rate': 2.423469387755102e-06, 'epoch': 19.14}
{'loss': 0.2802, 'grad_norm': 0.8907191753387451, 'learning_rate': 2.295918367346939e-06, 'epoch': 19.19}
{'loss': 0.2954, 'grad_norm': 1.0428447723388672, 'learning_rate': 2.1683673469387757e-06, 'epoch': 19.23}
{'loss': 0.2824, 'grad_norm': 0.7810648679733276, 'learning_rate': 2.040816326530612e-06, 'epoch': 19.28}
{'loss': 0.2777, 'grad_norm': 0.9300900101661682, 'learning_rate': 1.91326530

  0%|          | 0/134 [00:00<?, ?it/s]

{'eval_loss': 0.25163358449935913, 'eval_accuracy': 0.895315599113289, 'eval_runtime': 30.4632, 'eval_samples_per_second': 1125.424, 'eval_steps_per_second': 4.399, 'epoch': 20.0}
{'train_runtime': 9292.4387, 'train_samples_per_second': 121.538, 'train_steps_per_second': 0.476, 'train_loss': 0.34099182462260735, 'epoch': 20.0}
./data/llm/graph_prompter_hf_frozen/attentions.npy ./data/llm/graph_prompter_hf_frozen/hidden_states.npy ./data/llm/graph_prompter_hf_frozen/tokens.csv
train Forward Epoch 1 from 1


BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


test Forward Epoch 1 from 1
val Forward Epoch 1 from 1


# Post-Training
After Training, we load all attentions and hidden states from before and append the new fields to the dataset.

In [7]:
vanilla_df = ClassifierBase.read_forward_dataset("./data/llm/vanilla")
prompt_df = ClassifierBase.read_forward_dataset("./data/llm/prompt")
graph_prompter_hf_df = ClassifierBase.read_forward_dataset(
    "./data/llm/graph_prompter_hf"
)

In [10]:
kg_manager.llm_df

,source_id,target_id,id_x,id_y,prompt_feature_title,prompt_feature_genres,labels,split,prompt,gnn_feature_(no genres listed),...,gnn_feature_IMAX,gnn_feature_Musical,gnn_feature_Mystery,gnn_feature_Romance,gnn_feature_Sci-Fi,gnn_feature_Thriller,gnn_feature_War,gnn_feature_Western,graph_prompter_hf_source_embedding,graph_prompter_hf_target_embedding
0,0,5,0.0,5.0,Heat (1995),"['Action', 'Crime', 'Thriller']",1,train,"0[SEP]5[SEP]Heat (1995)[SEP]['Action', 'Crime'...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[0.6693655252456665, 0.4152011275291443, -0.31...","[0.5493870973587036, -0.5692430734634399, -0.6..."
1,0,89,0.0,89.0,Bottle Rocket (1996),"['Adventure', 'Comedy', 'Crime', 'Romance']",1,train,0[SEP]89[SEP]Bottle Rocket (1996)[SEP]['Advent...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[0.6030164957046509, 0.37829291820526123, -0.3...","[0.08589676022529602, -0.4689454734325409, 0.0..."
2,0,97,0.0,97.0,Braveheart (1995),"['Action', 'Drama', 'War']",1,train,"0[SEP]97[SEP]Braveheart (1995)[SEP]['Action', ...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[0.6387443542480469, 0.3706628084182739, -0.32...","[0.04940161854028702, -0.47019216418266296, -0..."
3,0,184,0.0,184.0,Billy Madison (1995),['Comedy'],1,train,0[SEP]184[SEP]Billy Madison (1995)[SEP]['Comedy'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[0.49271589517593384, 0.4562503397464752, -0.2...","[0.6636490821838379, -0.3091672956943512, 0.01..."
4,0,224,0.0,224.0,Star Wars: Episode IV - A New Hope (1977),"['Action', 'Adventure', 'Sci-Fi']",1,train,0[SEP]224[SEP]Star Wars: Episode IV - A New Ho...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[0.6498588919639587, 0.43191879987716675, -0.3...","[-0.08320911973714828, -0.060460180044174194, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125032,79,8846,NaN,NaN,Lovesick (2014),"['Comedy', 'Romance']",0,val,"79[SEP]8846[SEP]Lovesick (2014)[SEP]['Comedy',...",0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,"[-0.5735506415367126, -0.044633716344833374, -...","[0.47228026390075684, -0.911340594291687, 0.00..."
125033,300,2595,NaN,NaN,Dersu Uzala (1975),"['Adventure', 'Drama']",0,val,300[SEP]2595[SEP]Dersu Uzala (1975)[SEP]['Adve...,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.006543345749378204, -0.3807082176208496, 0....","[0.36955565214157104, -0.21707147359848022, -0..."
125034,321,59,NaN,NaN,Lawnmower Man 2: Beyond Cyberspace (1996),"['Action', 'Sci-Fi', 'Thriller']",0,val,321[SEP]59[SEP]Lawnmower Man 2: Beyond Cybersp...,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,"[-0.3852957785129547, 0.02625414729118347, -0....","[0.38845473527908325, -0.27163609862327576, -0..."
125035,388,2639,NaN,NaN,All the Vermeers in New York (1990),"['Comedy', 'Drama', 'Romance']",0,val,388[SEP]2639[SEP]All the Vermeers in New York ...,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,"[0.26095038652420044, 0.08665543794631958, -0....","[1.6701771020889282, -0.6786400079727173, 0.87..."


In [8]:
dataset = kg_manager.generate_huggingface_dataset(
    [vanilla_df, prompt_df, graph_prompter_hf_df, graph_prompter_hf_frozen_df],
    ["vanilla", "prompt", "graph_prompter_hf", "graph_prompter_hf_frozen_df"],
)

AssertionError: 

In [ ]:
dataset

NameError: name 'dataset' is not defined

In [ ]:
dataset.save_to_disk("./data/dataset.hf")

Saving the dataset (0/10 shards):   0%|          | 0/56469 [00:00<?, ? examples/s]

Saving the dataset (0/6 shards):   0%|          | 0/34284 [00:00<?, ? examples/s]

Saving the dataset (0/6 shards):   0%|          | 0/34284 [00:00<?, ? examples/s]

In [ ]:
dataset.push_to_hub("AhmadPython/MovieLens_KGE")

Uploading the dataset shards:   0%|          | 0/10 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/2.80k [00:00<?, ?B/s]

c:\Users\MARS\.conda\envs\hauptprojekt\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\MARS\.cache\huggingface\hub\datasets--AhmadPython--MovieLens_KGE. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/datasets/AhmadPython/MovieLens_KGE/commit/70c96f660429150e55b6437d59ee39da76891de7', commit_message='Upload dataset', commit_description='', oid='70c96f660429150e55b6437d59ee39da76891de7', pr_url=None, pr_revision=None, pr_num=None)